<a href="https://colab.research.google.com/github/KostasTheOne/Mobile-Apps-Project/blob/main/Profitable_Apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Profitable App Analysis for the App Store and Google Play Markets


Our aim in this project is to find mobile app profiles that are profitable for the App Store and Google Play markets. We're working as data analysts for a company that builds Android and iOS mobile apps, and our job is to enable our team of developers to make data-driven decisions with respect to the kind of apps they build.

At our company, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means that our revenue for any given app is mostly influenced by the number of users that use our app. Our goal for this project is to analyze data to help our developers understand what kinds of apps are likely to attract more users.

# Opening andExploring the Data

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play.
Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:

A [data set](https://www.kaggle.com/datasets/lava18/google-play-store-apps
) containing data about approximately ten thousand Android apps from Google Play.

A [data set](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps
) containing data about approximately seven thousand iOS apps from the App Store.

In [ ]:
import pandas as pd

In [ ]:
apple_data = pd.read_csv("/content/AppleStore.csv")
apple_data.head()

In [ ]:
apple_data.describe()

In [ ]:
apple_data.shape

In [ ]:
android_data = pd.read_csv("/content/googleplaystore.csv")
android_data.head()

In [ ]:
android_data.describe()

As we can see, there are some irregularities in our data. For instance, the maximum value in the Rating column is 19, which is clearly incorrect since ratings should range from 0 to 5. Additionally, the output of the describe() function only shows the Rating column, indicating that it is the only numeric column in the dataset. To make our analysis more meaningful, we will attempt to convert other columns—such as Reviews, Size, Installs, and Price—into numeric formats where appropriate.

In [ ]:
android_data.dtypes

In [ ]:
android_data.shape

We will convert the Reviews column to numeric values and then verify the changes by using the describe() function again.

In [ ]:
android_data["Reviews"] = pd.to_numeric(android_data["Reviews"], errors="coerce")
android_data.describe()


If we attempt to convert the Installs column to numeric, a ValueError will occur, indicating that the string "Free" cannot be converted to a number. This shows that the column contains non-numeric values, which must be identified and removed from the dataset before conversion.

In [ ]:
android_data["Installs"] = android_data["Installs"].str.replace(",", "", regex=True)

In [ ]:
android_data["Installs"] =android_data["Installs"].str.replace(r"\+", "", regex=True).astype(int)

We identify rows in the 'Installs' column that still contain non-numeric values
even after removing commas and plus signs. We use str.isnumeric() to check
which entries are purely numeric. The tilde (~) negates the condition, so
we select rows that are NOT numeric. We then print the app name, installs,
and type columns to inspect the problematic entries.

In [ ]:
non_numeric_installs = android_data[~android_data["Installs"].str.replace(",", "").str.replace(r"\+", "", regex=True).str.isnumeric()]
print(non_numeric_installs[["App", "Installs", "Type"]])


Then, we observe the problematic row from our data.

In [ ]:
android_data.loc[10472]

This row contains incorrect values and explains the observations we noticed at the start of our Google Play dataset analysis. So we remove the entire row and reset our dataset's index.

In [ ]:
android_data.drop(index=10472, inplace=True)

In [ ]:
android_data.reset_index(drop=True, inplace=True)

In [ ]:
android_data.shape

In [ ]:
android_data["Installs"] =android_data["Installs"].str.replace(r"\+", "", regex=True).astype(int)

In [ ]:
print(android_data["Installs"])

In [ ]:
android_data.describe()

In [ ]:
android_data["Price"] = android_data["Price"].str.replace(r"\$", "", regex=True).astype(float)

In [ ]:
android_data.duplicated().sum()

In [ ]:
android_data[android_data.duplicated()]

In [ ]:
apple_data.duplicated().sum()

We observe that the Android dataset contains duplicate entries, whereas the Apple dataset appears to be clean. It is important to clearly define what we mean by duplicate values. Using the code above, we identify rows that are identical across all columns. However, there may also be apps that share the same name but have different values in other columns, and these are not captured by this definition of duplicates.

In [ ]:
duplicated_values=android_data[android_data.duplicated(subset=["App"], keep=False)]

In [ ]:
print(duplicated_values)

In [ ]:
android_data[android_data['App']=="Instagram"]

In [ ]:
duplicated_apps = []
unique_apps = []

for app in android_data["App"]:
  if app in unique_apps:
    duplicated_apps.append(app)
  else:
    unique_apps.append(app)
print(len(duplicated_apps))
print(duplicated_apps[:10])

We should delete the duplicates but not random. As we can see in the "Instagram" example the only difference is in the number of reviews in each row. It's like they updated the dataset in different times, so we are going to keep only the rows with the most reviews, which means we are keeping the latest addition.

The first step is to use the groupby() function to group rows by the "App" column. From each group, we select the "Reviews" column, and then we apply the .max() function to retain only the maximum number of "Reviews" for each app. As a result, we obtain the reviews_max Series, which contains one value per app corresponding to its highest number of reviews.

In [ ]:
reviews_max = android_data.groupby("App")["Reviews"].max()
reviews_max.head()

In [ ]:
android_clean = android_data.loc[android_data.groupby("App")["Reviews"].idxmax()]

In a previous code cell, we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and 1,181.

In [ ]:
print("Expected lenght:",len(android_data) - 1181)
print("Cleaned data lenght:",len(android_clean))

In [ ]:
android_clean[android_clean['App']=="Instagram"]

In [ ]:
android_clean.reset_index(drop=True, inplace=True)

In [ ]:
android_clean.head()

In [ ]:
android_clean.describe()

In [ ]:
android_clean.shape

We do the same for apple_data.

In [ ]:
duplicated_values=apple_data[apple_data.duplicated(subset=["track_name"], keep=False)]

In [ ]:
print(duplicated_values)

In [ ]:
duplicate_count = apple_data["track_name"].duplicated().sum()
print(duplicate_count)
duplicate_apps = apple_data[apple_data["track_name"].duplicated(keep=False)]["track_name"].unique()
print(duplicate_apps)

In [ ]:
apple_data[apple_data["track_name"]=="Mannequin Challenge"]
apple_data[apple_data["track_name"]=="VR Roller Coaster"]

In [ ]:
apple_reviews_max = apple_data.groupby("track_name")["rating_count_tot"].max()
apple_clean = apple_data.loc[apple_data.groupby("track_name")["rating_count_tot"].idxmax()]
print(len(apple_data)-2)
print(len(apple_clean))

# Removing Non-English Apps

If you explore the datasets, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. We're not interested in keeping these kind of apps, so we'll remove them.

We make a function that detects non-English characters in app names. Since some English-name apps contain emojis or a small number of non-ASCII characters we allow up to 3 non-ASCII characters before classifying an app name as non-English. This approach is not perfectly accurate, it is appropriate for this dataset and helps minimize unnecessary data loss.

In [ ]:
def is_english(string):
  non_ascii = sum(1 for char in string if ord(char) > 127)
  ascii_chars = sum(1 for char in string if ord(char) <= 127)
  if non_ascii >3:
    return False
  else:
    return True

In [ ]:
android_english_apps = android_clean["App"].apply(is_english)

In [ ]:
android_english= android_clean[android_english_apps]

In [ ]:
android_english.shape

In [ ]:
android_english.head()

In [ ]:
apple_english_apps = apple_clean["track_name"].apply(is_english)

In [ ]:
apple_english= apple_clean[apple_english_apps]

In [ ]:
apple_english.shape

In [ ]:
apple_english.head()

# Isolating the Free Apps

As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps, and we'll need to isolate only the free apps for our analysis. Below, we isolate the free apps for both our data sets.

In [ ]:
android_final = android_english[android_english["Price"]==0]
print(len(android_final))

In [ ]:
apple_final = apple_english[apple_english["price"]==0]
print(len(apple_final))

# Most common Genres by App

As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we then develop it further.
If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

In [ ]:
android_final["Category"].value_counts()

In [ ]:
genre_percentage = android_final["Category"].value_counts(normalize=True)
genre_percentage = (genre_percentage * 100).round(2)
print(genre_percentage)

We continue by examining the frequency table for the prime_genre column of the App Store data set.

In [ ]:
apple_final["prime_genre"].value_counts()

In [ ]:
apple_genre = apple_final["prime_genre"].value_counts(normalize=True)
apple_genre = (apple_genre*100).round(2)
print(apple_genre)

# Most Popular Apps by Genre on the App Store

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

In [ ]:
apple_final.groupby("prime_genre")["rating_count_tot"].mean().sort_values(ascending=False)

On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [ ]:
apple_final.loc[apple_final["prime_genre"]=="Navigation", ["track_name","rating_count_tot"]].sort_values(by="rating_count_tot", ascending=False)

The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings, while the other apps may struggle to get past the 10,000 threshold. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages, but we'll leave this level of detail for later.

Reference apps have 74,942 user ratings on average, but it's actually the Bible and Dictionary.com which skew up the average rating:

In [ ]:
apple_final.loc[apple_final["prime_genre"]=="Reference", ["track_name","rating_count_tot"]].sort_values(by="rating_count_tot", ascending=False)

However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.

Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.

Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.

Now let's analyze the Google Play market a bit.

# Most Popular Apps by Genre on Google Play

For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity.We've already cleaned our data from the '+' symbol. One problem with this data is that is not precise. For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to get an idea which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on.

In [ ]:
android_final.groupby("Category")["Installs"].mean().sort_values(ascending=False)

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [ ]:
android_final.loc[android_final["Category"]=="COMMUNICATION",["App", "Installs"]].sort_values(by="Installs", ascending=False)

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [ ]:
under_100m = android_final.loc[
    (android_final["Category"]=="COMMUNICATION") &
    (android_final["Installs"] < 100000000)]
print(under_100m["Installs"].mean())

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

Let's take a look at some of the apps from this genre and their number of installs:

In [ ]:
android_final.loc[android_final["Category"] == "BOOKS_AND_REFERENCE", ["App", "Installs"]].sort_values(by="Installs", ascending=False)

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [ ]:
android_final.loc[(android_final["Category"] == "BOOKS_AND_REFERENCE") &
                 (android_final["Installs"]>=100000000),["App", "Installs"]].sort_values(by="Installs", ascending=False)


However, it looks like there are only a few very popular apps, so this market still shows potential. Let's try to get some app ideas based on the kind of apps that are somewhere in the middle in terms of popularity (between 1,000,000 and 100,000,000 downloads):

In [ ]:
android_final.loc[(android_final["Category"] == "BOOKS_AND_REFERENCE") &
                 (android_final["Installs"]<100000000) &
                 (android_final["Installs"]>=1000000),["App", "Installs"]].sort_values(by="Installs", ascending=False)

This niche seems to be dominated by software for processing and reading ebooks, as well as various collections of libraries and dictionaries, so it's probably not a good idea to build similar apps since there'll be some significant competition.

We also notice there are quite a few apps built around the book Quran, which suggests that building an app around a popular book can be profitable. It seems that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets.

However, it looks like the market is already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.

# Conclusions

In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.